In [1]:
# !pip install unsloth
# !pip install huggingface
# !pip install tf-keras

In [2]:
import torch
from transformers import BitsAndBytesConfig
from unsloth import FastLanguageModel

fourbit_models = [
    "unsloth/Qwen3-1.7B-unsloth-bnb-4bit", # Qwen 14B 2x faster
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",

    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
] # More models at https://huggingface.co/unsloth
model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,  # or torch.float16 if no bf16
    llm_int8_enable_fp32_cpu_offload=True,  # key for offloading
)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=8048,          # consider lowering if you still OOM
    quantization_config=bnb_config,
    device_map="auto",            # let HF shard between GPU/CPU
)


==((====))==  Unsloth 2025.12.9: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla V100-SXM2-16GB. Num GPUs = 1. Max memory: 15.766 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [3]:
import pandas as pd
import os
datafold = "/sbgenomics/project-files/"
file = "y_pasc_score2024.csv"
# file = "RECOVERAdult_BiostatsDerived_202412_symptoms_deID.csv"
result = pd.read_csv(datafold + file)
result = result.rename(columns={"PARTICIPANT_ID":"id", "pasc_score_2024":"name"})
result['index'] = "pasc_score_2024"
result_reordered = result.iloc[:, [0, 3,1,2]]
# file = "all_methods_summary.csv"
file = "vaccine_2026.csv"
df = pd.read_csv(datafold + file)
df=df.iloc[:, [1, 2, 3,4]]
combined_df = pd.concat([result_reordered, df], ignore_index=True, axis=0)


In [4]:
combined_df.head()

,id,index,date,name
0,RA11305,pasc_score_2024,2021-01-01,16.0
1,RA11305,pasc_score_2024,2021-04-03,14.0
2,RA11305,pasc_score_2024,2021-06-28,8.0
3,RA11305,pasc_score_2024,2021-09-28,13.0
4,RA11305,pasc_score_2024,2021-12-23,10.0


In [14]:
import numpy as np
# B = "pasc_score_2024"  # set your target value
# out = combined_df.groupby("id").filter(lambda g: (g["index"] == B).sum() > 4)
# print(out.shape)#282581, 4)

ids = np.unique(out['id'])
print(len(ids)) #15158 13451
index = 5

selected_ids = [id_value for id_value in ids[5:20]] # Make sure ids is just the list of values
output_ind_df = combined_df[combined_df['id'].isin(selected_ids)]

# output_ind_df = combined_df[combined_df['id']==ids[index]]
print(output_ind_df.shape)
# df_new = output_ind_df.drop(columns=['id'])
output_ind = output_ind_df.to_string(index=True, header=True)
# output_string = combined_df.to_string(index=True, header=True)

13451
(313, 4)


In [18]:
# output_ind_df
prompt = (
"""You are a clinical, medical, physician, and statistical expert.

You will be given a table for several patient with columns:
- id: patient id
- date: date of record
- index: 0=enrollment; 1/2/3/4=vaccine dose number; "followup" or "followup_k"=follow-up visit, pasc=wellness score (lower is better). Threshold: >=12 = Long COVID (PASC), <12 = No PASC.
- name: free-text description (may include vaccine brand like pfizer/moderna/etc, and may contain notes, if it is value, then pasc)
- (optional) followup_2: "no" means no vaccine between this visit and the previous visit; "yes" means a vaccine occurred between visits.

Important notes:
- Vaccines may occur before enrollment (index=0).
- Dates may be out of order in the raw text; sort by date when building a timeline.
- If vaccine brand is ambiguous/misspelled (e.g., "pfzier"), normalize to the closest common brand and also keep the raw string.

TASKS
Write a concise summary,sStart a section exactly titled: #Summary
   In #Summary include:
   - 3–5 sentences describing the patients’ course, key events, and strongest observed associations of the longitudal information

INPUT TABLE :
"""
+ output_ind +
"""
"""
)


In [19]:
# # prompt = "here is the vaccine record of one patient, can you provide a summary of the useful information " +output_string + " and present results in a table to be saved, such as date of vaccine, index of vaccine, name of vaccine"
# prompt = "here is followup of maybe long covid patient, vaccines may be taken before enrollment. there are 4 columns, id column: is the patient id, date column: is the date of item recorded, index column: 0 means the enrollment, 1,2,3,4 means the first, second, third and fourth vaccine, followup means the followup, pasc_score_2024 means the score of wellness, the lower the score, the better the patient, 12 is usually the threshold, >=12 means long covid, < 12 means no; name column: is the description, such as pfzier is the vaccine name, number with pasc_score_2024 means the score, with index column followup_2, no means there is no vaccine between this visit and previous visit, yes means another vaccine " +\
# output_ind + " make analysis of the table, then find some discoveries of such as pasc score and the vaccine, the pattern, the timeline, when peak happens, what are the events before and after, discories that usually in nature/science. Write summary in #Summary"

messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0.7,
    max_new_tokens = 3024,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

<|im_start|>user
You are a clinical, medical, physician, and statistical expert.

You will be given a table for several patient with columns:
- id: patient id
- date: date of record
- index: 0=enrollment; 1/2/3/4=vaccine dose number; "followup" or "followup_k"=follow-up visit, pasc_score_2024=wellness score (lower is better). Threshold: >=12 = Long COVID (PASC), <12 = No PASC.
- name: free-text description (may include vaccine brand like pfizer/moderna/etc, and may contain notes, if it is value, then pasc_score_2024)
- (optional) followup_2: "no" means no vaccine between this visit and the previous visit; "yes" means a vaccine occurred between visits.

Important notes:
- Vaccines may occur before enrollment (index=0).
- Dates may be out of order in the raw text; sort by date when building a timeline.
- If vaccine brand is ambiguous/misspelled (e.g., "pfzier"), normalize to the closest common brand and also keep the raw string.

TASKS
Write a concise summary,sStart a section exactly tit

Unsloth: Input IDs of shape torch.Size([1, 11669]) with length 11669 > the model's max sequence length of 8048.
We shall truncate it ourselves. It's imperative if you correct this issue first.


OutOfMemoryError: CUDA out of memory. Tried to allocate 80.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 43.44 MiB is free. Process 59509 has 15.72 GiB memory in use. Of the allocated memory 15.14 GiB is allocated by PyTorch, and 196.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# # # pip install accelerate
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")
# model = AutoModelForCausalLM.from_pretrained("google/gemma-7b", device_map="auto")

# input_text = "Write me a poem about Machine Learning."
# input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

# outputs = model.generate(**input_ids)
# print(tokenizer.decode(outputs[0]))
### **Conclusion**
- Vaccines (Pfizer/Moderna) may help reduce long COVID symptoms (PASC scores ≤12), but outcomes vary by individual.  
- Peaks in PASC scores often occur 6–12 months post-enrollment, followed by recovery in many cases.  
- Long-term followups (≥3 years) show that some patients achieve sustained recovery, even without additional vaccines.  

This analysis highlights the complex interplay between vaccination, immune response, and long-term health outcomes in post-acute sequelae of SARS-CoV-2 infection.<|im_end|>

In [72]:
import transformers
# encoding/encoding_dsv32.py
from encoding_dsv32 import encode_messages, parse_message_from_completion_text

tokenizer = transformers.AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-V3.2")

messages = [
    {"role": "user", "content": "hello"},
    {"role": "assistant", "content": prompt},#"Hello! I am DeepSeek.", "reasoning_content": "thinking..."},
    {"role": "user", "content": "1+1=?"}
]
encode_config = dict(thinking_mode="thinking", drop_thinking=True, add_default_bos_token=True)

# messages -> string
prompt = encode_messages(messages, **encode_config)
# Output: "<｜begin▁of▁sentence｜><｜User｜>hello<｜Assistant｜></think>Hello! I am DeepSeek.<｜end▁of▁sentence｜><｜User｜>1+1=?<｜Assistant｜><think>"

# string -> tokens
tokens = tokenizer.encode(prompt)
# Output: [0, 128803, 33310, 128804, 128799, 19923, 3, 342, 1030, 22651, 4374, 1465, 16, 1, 128803, 19, 13, 19, 127252, 128804, 128798]


ModuleNotFoundError: No module named 'encoding_dsv32'

In [ ]:
**PASC Score Fluctuations Over Time**:
**Possible Biological Insights**:
   - **Waning Immunity**: The peak in 2024, 20 months after the last vaccine, may reflect **diminished protection** from prior doses, leading to a resurgence of symptoms.
   - **Immune Memory and Long COVID**: The patient’s PASC score fluctuated, suggesting **variable immune response**. The 2024 peak might indicate **reduced immune memory** or **viral reactivation** (e.g., SARS-CoV-2 variants or other pathogens).
   - **Seasonal or Environmental Factors**: The spike in 2024 could correlate with **seasonal viral activity** (e.g., winter 2023–2024) or **environmental stressors**.

5. **Clinical Implications**:
   - **Long-Term Monitoring**: The case highlights the need for **long-term monitoring** of patients post-vaccination, especially those with prior long COVID.
   - **Vaccine Efficacy Over Time**: The peak in 2024 suggests that **vaccine protection may not fully prevent long-term symptoms**, especially in vulnerable individuals.
   - **Therapeutic Window**: The drop in PASC score to 2.0 in May 2024 suggests **possible recovery** or **intervention** (not documented here), indicating variability in disease progression.


                                                           ### **6. Scientific Insights**
- **Long COVID is not static**:  
  - Patients may experience **remissions and relapses**, even with prior vaccinations.  
  - **PASC scores are not predictive of long-term outcomes** without additional biomarkers.  

- **Vaccine Efficacy for Long COVID**:  
  - While vaccines may reduce acute symptoms, **they do not eliminate Long COVID risk** in all individuals.  

- **Need for Longitudinal Studies**:  
  - More data on **vaccine timing, booster doses, and comorbidities** are needed to understand Long COVID dynamics.  

---

### **7. Recommendations for Further Research**
- Investigate **seasonal triggers** (e.g., viral infections, pollution) in Long COVID relapses.  
- Explore **biomarkers** (e.g., cytokine profiles, autoantibodies) to predict PASC score fluctuations.  
- Study **vaccine boosters** in Long COVID patients to assess sustained protection.  
- Address **missing data** in follow-up visits to improve outcome prediction models.  

---

### **Summary**
Patient RA16050’s PASC scores demonstrate **significant variability**, with a **sharp initial improvement** post-enrollment, followed by **recurrent Long COVID symptoms**. Pre-vaccination history and the absence of further vaccines suggest that **Long COVID is not fully mitigated by vaccination alone**. The **peak in September 2023** highlights the need for ongoing monitoring and personalized interventions for Long COVID patients.<|im_end|>

In [30]:
from transformers import AutoTokenizer

model_name = 'nvidia/Nemotron-Cascade-8B'
tokenizer = AutoTokenizer.from_pretrained(model_name)

'''
single-turn example
'''
messages = [
    {"role": "user", "content": "calculate 1+1?"}
]

# thinking mode
prompt_thinking = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=True)
# prompt_thinking = '<|im_start|>system\nYou are a helpful and harmless assistant.<|im_end|>\n<|im_start|>user\ncalculate 1+1? /think<|im_end|>\n<|im_start|>assistant\n'

# instruct mode
prompt_instruct = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)
# prompt_instruct = '<|im_start|>system\nYou are a helpful and harmless assistant.<|im_end|>\n<|im_start|>user\ncalculate 1+1? /no_think<|im_end|>\n<|im_start|>assistant\n'


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]